# Execute embedding and search on remote server

In [1]:
!pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 76.6 MB/s eta 0:00:00


## Transfer data file

Configuration file for embedding and search

In [5]:
config={"textfile":"test.dat.gz", # always a file finishing by .dat.gz (jsonl compressed)
        "mode": "semsearch sdgbert",
        "classes": "goals" # goals, targets, both
       }

In [7]:
import getpass
import subprocess
import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
import json

# Your code to be executed when the button is clicked
def send_data_to_server():
    # Load environment variables from .env file
    load_dotenv()
    mode="sdgbert" #or semsearch
    # Define a variable based on a value from .env
    server = os.getenv("server")
    username=os.getenv("username")
    remote_host=username+"@"+server
    filename = config["textfile"]
    filename3="searchconfig.json"
    print(remote_host)
    with open('searchconfig.json', "w") as f:
        json.dump(config, f, indent=4)
          
    remote_path = "/home/codechallenge/InnoClass/backend/classEngine/data/"
    #password = getpass.getpass(f"Enter password for {remote_host}: ")
    #command = ["scp", "-o", f"StrictHostKeyChecking=no", "-o", f"PreferredAuthentications=password", filename, f"{remote_host}:{remote_path}"]
    command = ["scp", filename, f"{remote_host}:{remote_path}"]
    command3 = ["scp", filename3, f"{remote_host}:{remote_path}"]
    try:
        print("command 1 : ")
        process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate() # Append newline to password
        return_code = process.returncode
        if return_code == 0:
            print("SCP command executed successfully.")
            print("Output:", stdout)
        else:
            print(f"SCP command failed with error (code {return_code}):")
            print("Error!:", stderr)
        print("command2 : ")
        process = subprocess.Popen(command3, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate() # Append newline to password
        return_code = process.returncode
    
        if return_code == 0:
            print("SCP command executed successfully.")
            print("Output:", stdout)
        else:
            print(f"SCP command failed with error (code {return_code}):")
            print("Error!:", stderr)
    
    except FileNotFoundError:
        print("Error: scp command not found.")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    print("Sending data to server...")


# Create the button
send_button = widgets.Button(description="Send data to Server")
display(send_button)
# Get list of files ending with ".sen.gz" from current directory
files = [f for f in os.listdir('.') if f.endswith('dat.gz')]
files2 = [f for f in os.listdir('.') if f.endswith('.dat')]

# Define the button's click event handler
def on_button_click(b):  
    send_data_to_server()

# Attach the event handler to the button's click event
send_button.on_click(on_button_click)

Button(description='Send data to Server', style=ButtonStyle())

In [3]:
!pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 69.2 MB/s eta 0:00:00


In [4]:
import paramiko
import os
from dotenv import load_dotenv
load_dotenv()


def execute_remote_command(hostname, username, private_key_path, command):
    """Executes a command on a remote machine using SSH keys."""
    try:
        private_key = paramiko.Ed25519Key.from_private_key_file(private_key_path)
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname, username=username, pkey=private_key)

        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        error = stderr.read().decode()

        client.close()

        if error:
            print(f"Error: {error}")
        return output

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

execute_button = widgets.Button(description="Execute Search on Server")
# Define the button's click event handler
def on_execute_button_click(b):
    hostname = os.getenv("server")
    username = os.getenv("username")
    private_key_path = os.getenv("key")
    command = "cd /home/codechallenge/InnoClass/backend/; docker-compose up embedsearch"  # Replace with your desired command
    result = execute_remote_command(hostname,username, private_key_path,command)
    
    if result:
        print("Command output:")
        print(result)

# Attach the event handler to the button's click event
execute_button.on_click(on_execute_button_click)
display(execute_button)


Button(description='Execute Search on Server', style=ButtonStyle())